# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import requests
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weather_df = "../WeatherPy/output_data/cities.csv"
weather = pd.read_csv(weather_df)
weather

,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortuna,40.5982,-124.1573,52.00,87,90,19.57,US,1612568167
1,1,caravelas,-17.7125,-39.2481,80.20,80,97,14.56,BR,1612568135
2,2,ushuaia,-54.8000,-68.3000,46.40,76,75,25.32,AR,1612568062
3,3,nikolskoye,59.7035,30.7861,12.00,78,75,11.18,RU,1612568167
4,4,seoul,37.5683,126.9778,39.20,81,75,6.91,KR,1612568164
...,...,...,...,...,...,...,...,...,...,...
582,582,newport,51.5877,-2.9984,37.99,88,1,1.01,GB,1612568251
583,583,rahatgarh,23.7833,78.3667,52.81,64,6,6.33,IN,1612568251
584,584,la ronge,55.1001,-105.2842,-18.40,63,94,12.66,CA,1612568251
585,585,bandarbeyla,9.4942,50.8122,73.47,71,17,8.95,SO,1612568054


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)

In [10]:
locations = weather[["Lat", "Lng"]].astype(float)
humidity = weather["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
cities_weather = weather.loc[(weather["Temp"]>70) & (weather["Temp"]<80) & (weather["Wind Speed"]<10) & (weather["Cloudiness"]==0)]
cities_weather

ideal_weather = cities_weather.dropna()
ideal_weather = cities_weather.reset_index()
ideal_weather




,index,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,9,9,saint-joseph,-21.3667,55.6167,78.80,65,0,6.91,RE,1612568168
1,34,34,mar del plata,-38.0023,-57.5575,73.00,49,0,5.75,AR,1612567879
2,53,53,san ciro de acosta,21.6333,-99.8167,74.37,54,0,8.61,MX,1612568175
3,95,95,moron,-34.6534,-58.6198,73.99,63,0,1.99,AR,1612567883
4,96,96,extremoz,-5.7056,-35.3072,78.80,78,0,5.75,BR,1612568182
5,115,115,upington,-28.4478,21.2561,71.60,23,0,4.61,ZA,1612568184
6,142,142,santiago del estero,-27.7951,-64.2615,75.65,40,0,5.26,AR,1612568188
7,205,205,presidente epitacio,-21.7633,-52.1156,71.83,77,0,6.04,BR,1612568197
8,218,218,buin,-33.7333,-70.7500,75.99,56,0,5.75,CL,1612568198
9,224,224,saint-philippe,-21.3585,55.7679,78.80,65,0,6.91,RE,1612568199


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = ideal_weather
hotel_df["Hotel Name"] = np.nan
hotel_df


,index,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,9,9,saint-joseph,-21.3667,55.6167,78.80,65,0,6.91,RE,1612568168,NaN
1,34,34,mar del plata,-38.0023,-57.5575,73.00,49,0,5.75,AR,1612567879,NaN
2,53,53,san ciro de acosta,21.6333,-99.8167,74.37,54,0,8.61,MX,1612568175,NaN
3,95,95,moron,-34.6534,-58.6198,73.99,63,0,1.99,AR,1612567883,NaN
4,96,96,extremoz,-5.7056,-35.3072,78.80,78,0,5.75,BR,1612568182,NaN
5,115,115,upington,-28.4478,21.2561,71.60,23,0,4.61,ZA,1612568184,NaN
6,142,142,santiago del estero,-27.7951,-64.2615,75.65,40,0,5.26,AR,1612568188,NaN
7,205,205,presidente epitacio,-21.7633,-52.1156,71.83,77,0,6.04,BR,1612568197,NaN
8,218,218,buin,-33.7333,-70.7500,75.99,56,0,5.75,CL,1612568198,NaN
9,224,224,saint-philippe,-21.3585,55.7679,78.80,65,0,6.91,RE,1612568199,NaN


In [59]:
hotel_df = []

hotel = 0

for cities in hotel_df:
    x+= 1 
    target_radius = 5000
    target_type = "lodging"
    coords = str(ideal_weather["Lat"][x])+ str(ideal_weather["Lng"][x])


    params = {
        "location": coords,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try:
        response = requests.get(base_url, params=params).json()
        hotel_df.append({"Hotel Name": hotel['results'][0]['name']})
    except KeyError:
        pass

hotel_df

[]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
